# DEMO: Data Exploration and Feature Engineering

## Imports

In [32]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from snowflake.ml.modeling.preprocessing import *
from snowflake.ml.modeling.impute import *
import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Create Snowpark Session

In [33]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [34]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "MUSHROOMS_DB"."PUBLIC"
Current Warehouse: "MUSHROOM_M_WH"


## Snowpark DataFrames vs. Pandas DataFrames

In [35]:
# Creating a Pandas DataFrame
pandas_df = pd.read_csv('data/mushrooms.csv')
print(type(pandas_df))

<class 'pandas.core.frame.DataFrame'>


In [36]:
# Creating a Snowpark DataFrame
snowpark_df = session.table("MUSHROOM_DB.PUBLIC.MUSHROOMS") # add full path to table db.shema.table
print(type(snowpark_df))

<class 'snowflake.snowpark.table.Table'>


In [37]:
# Compare size
print('Size in MB of Pandas DataFrame in Memory:\n', np.round(sys.getsizeof(pandas_df) / (1024.0**2), 2))
print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))

Size in MB of Pandas DataFrame in Memory:
 10.34
Size in MB of Snowpark DataFrame in Memory:
 0.0


The only thing stored in a Snowpark DataFrame is the SQL needed to return data

In [38]:
snowpark_df.queries

{'queries': ['SELECT  *  FROM (MUSHROOM_DB.PUBLIC.MUSHROOMS)'],
 'post_actions': []}

A Snowpark DataFrame can be converted to a Pandas DataFrame. This will pull the data from Snowflake into the Python enviroment memory.

In [39]:
pandas_df2= snowpark_df.to_pandas()

Both our Pandas DataFrames has the same shape

In [40]:
pandas_df.shape, pandas_df2.shape

((8124, 23), (8124, 23))

Showing a Snowpark DataFrame

In [41]:
pandas_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [42]:
# check for null vlaues
pandas_df2.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [43]:
#show all cols
pandas_df2.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [44]:
# snowpark_df.show() <- also possible
snowpark_df.limit(5).show() # <- collects first 5 rows and displays as pandas-dataframe

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"class"  |"cap-shape"  |"cap-surface"  |"cap-color"  |"bruises"  |"odor"  |"gill-attachment"  |"gill-spacing"  |"gill-size"  |"gill-color"  |"stalk-shape"  |"stalk-root"  |"stalk-surface-above-ring"  |"stalk-surface-below-ring"  |"stalk-color-above-ring"  |"stalk-color-below-ring"  |"veil-type"  |"veil-color"  |"ring-number"  |"ring-type"  |"spore-print-color"  |"population"  |"habitat"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Simple Transformations
Select specific columns

In [45]:
snowpark_df.head()

AttributeError: Table object has no attribute head

In [ ]:
snowpark_df = snowpark_df.select('class')
#snowpark_df = snowpark_df[["CLASS"]] # -> pandas-like selection
snowpark_df.show()

SnowparkSQLException: (1304): 01b1b5be-0303-4b07-0000-0001ffea4135: 000904 (42000): SQL compilation error: error line 1 at position 7
invalid identifier 'CLASS'

To add a new column to a Snowpark DataFrame the **with_column** method can be used.  
In below example we are adding a neew column, AGE, that calculates the number of years that DAYS_BIRTH is.

In [ ]:
# Create a new column
# Formula: Absolute Value of DAYS_BIRTH divided by 365 days rounded down
snowpark_df = snowpark_df.with_column('AGE', F.floor(F.abs(F.col('DAYS_BIRTH')) / 365))
snowpark_df.show()

If we do not want to use specific columns we can use **drop** to remove those from a Snowpark DataFrame.  
**Note:** This is not removing them from the underlying table.

In [ ]:
# Drop a column
snowpark_df = snowpark_df.drop('DAYS_BIRTH')
snowpark_df.show()

To filter/select specific rows we use **filter**

In [ ]:
# Filter data
snowpark_df = snowpark_df.filter(F.col('NAME_INCOME_TYPE').in_(['Pensioner','Student']))
snowpark_df.show()

To aggregate data the **group_by** method are used in combination with the **agg** method.

In [ ]:
snowpark_df = snowpark_df.group_by(['CODE_GENDER','NAME_INCOME_TYPE']).agg([F.avg('AGE').as_('AVG_AGE')])
snowpark_df.show()

To sort the data in the dataframe **sort** is used.

In [ ]:
# Sort data
snowpark_df = snowpark_df.sort(F.col('AVG_AGE').desc())
snowpark_df.show()

## Simple Data Analysis
In this section we will use API Snowpark to do some basic analysis of our data.  
Start by creating a new Snowpark DataFrame

In [ ]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')

The **count** method on a DataFrame will return the number of rows

In [ ]:
# Number of rows in dataset
snowpark_df.count()

If we want to filter out duplicated rows, keeping only one, we can use the **drop_duplicates** method.

In [ ]:
# Lets drop duplicates based on ID
snowpark_df = snowpark_df.drop_duplicates('ID')
snowpark_df.count()

Duplicated rows are only filtered and we can see the logic for it by examining the SQL for the DataFrame, using ['queries'][0] will return the first SQL statement for the DataFrame

In [ ]:
print(snowpark_df.queries['queries'][0])

Using the **describe** method will return some basic statistics for all numeric and string columns.

In [ ]:
# Calculating various statistics per column
snowpark_df.describe().show()

Using **group_by** and **agg** alows us to calculate the mean value of AMT_INCOME_TOTAL by NAME_INCOME_TYPE and CODE_GENDER. Using **sort** to return the data ordered by NAME_INCOME_TYPE in ascending order and AVG_INCOME by descending order.

In [ ]:
# Average Income per Income Type and Gender
analysis_df = snowpark_df.group_by(['NAME_INCOME_TYPE','CODE_GENDER']).agg([F.mean('AMT_INCOME_TOTAL').as_('AVG_INCOME')])
analysis_df = analysis_df.sort('NAME_INCOME_TYPE', F.col('AVG_INCOME').desc())
analysis_df.show()

We can use the SimpleImputer in snowflake.ml.preprocessing to replace missing values with the most frequent.

In [ ]:
my_imputer = SimpleImputer(input_cols=['OCCUPATION_TYPE'], output_cols=['OCCUPATION_TYPE'] ,strategy='most_frequent')
my_imputer.fit(snowpark_df)
snowpark_df = my_imputer.transform(snowpark_df)
snowpark_df.describe().show()

The missing value handling logic is converted in SQL to *iff("OCCUPATION_TYPE" IS NULL, 'Laborers', "OCCUPATION_TYPE") AS "OCCUPATION_TYPE"*

In [ ]:
print(snowpark_df.queries['queries'][0])

A DataFrame always has a schema with all columns and the data types for them

In [ ]:
snowpark_df.schema

We can itirate through the schema to get the columns of specific data types

In [ ]:
# Get all categorical columns
categorical_types = [T.StringType]
categorical_columns = [c.name for c in snowpark_df.schema.fields if type(c.datatype) in categorical_types]
categorical_columns

We can then use those to loop through and get the number of unique values

In [ ]:
# Number of unique values per categorical column
unique_values = []
for column in categorical_columns:
    unique_values.append([column, snowpark_df.select(column).distinct().count()])
pd.DataFrame(unique_values, columns=['COLUMN_NAME','NUM_UNIQUE_VALUES'])

## Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.  
**save_as_table** saves the result in a table, if **mode='overwrite'** then it will also replace the data that is in it.

In [ ]:
snowpark_df.write.save_as_table(table_name='MY_FIRST_ANALYSIS', mode='overwrite')
session.table('MY_FIRST_ANALYSIS').show()

In [ ]:
session.close()